In [3]:
from pathlib import Path
import os
import sys

sys.path.append(Path(os.path.abspath('')).parent.parent)
os.chdir(Path(os.path.abspath('')).parent.parent)

import numpy as np
import pandas as pd 
import xarray as xr
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score

import config.settings as cset

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import geopandas as gpd 
from matplotlib import colormaps as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely import box

labelsize_small  = 13
labelsize_medium = 15
labelsize_large  = 17

### Step 1: Load and Prepare Data 

In [4]:
# loading & preparing gdp data
gdp_df   = pd.read_csv(cset.file_burke).loc[:, ['iso', 'scen', 'run', 'year', 'diff_SSP2', 'SSP2']].loc[:, ['iso', 'scen', 'run', 'year', 'diff_SSP2', 'SSP2']]
gdp_df.loc[gdp_df.scen == 'Ref1p5', 'scen'] = 'Ref'
# only keep data for year that we are interested in: 
gdp_baseline = gdp_df[gdp_df.year == 2015]['SSP2'].values
gdp_df   = gdp_df[(gdp_df.year == cset.gdp_target_year)].copy()
# adding for compatibility with characteristics df s
gdp_df['scen_run'] = gdp_df['scen'] + '_' + gdp_df['run'].astype(str)

# loading & preparing tas_baseline data
tas_df = pd.read_csv(Path.joinpath(cset.path_GDP, '2408-adj_countrylevel_total.csv'))
tas_df = tas_df[tas_df.year == 2015].copy()
tas_df = tas_df.rename(columns = {'run': 'scen_run', 'tas': 'tas_soc'})
tas_df = tas_df.dropna(subset=['tas_soc']).groupby('iso').filter(lambda x: x['tas_soc'].notna().all())
tas_df.loc[tas_df['scen_run'].str.contains('Ref_1p5'), 'scen_run'] = tas_df['scen_run'].str.replace('Ref_1p5', 'Ref')

# loading & preparing glmt data 
glmt_char_df = pd.read_csv(Path.joinpath(cset.path_MESMER_char, 'gmt_characteristics_thsld_150.csv'), index_col = 0)

# merge datasets into a single regression dataset
regression_df = gdp_df.loc[:, ['iso', 'scen_run', 'diff_SSP2']].merge(glmt_char_df, left_on='scen_run', right_index=True)
regression_df['GDP_baseline'] = gdp_baseline
regression_df = regression_df.merge(tas_df[['iso', 'scen_run', 'tas_soc']], on=['iso', 'scen_run'], how='left')
#   - add baseline temperature values
regression_df['tas_baseline'] = regression_df.groupby('iso')['tas_soc'].transform('mean')
regression_df['tas_soc']      -= regression_df['tas_baseline']
regression_df                 = regression_df.sort_values(['iso', 'scen_run'])
regression_df['gmt_os']       = regression_df['gmt_max']- regression_df['gmt_eoc']
regression_df['log_gmt_cum']  = np.log(regression_df['gmt_cum'])

regression_df['gmt_delta_eoc'] = 1.5+regression_df['gmt_exc']*2-regression_df['gmt_eoc']
regression_df['gmt_delta_max'] = regression_df['gmt_max']-(1.5+2*regression_df['gmt_exc'])
# clear storage
# del gdp_df, glmt_char_df, tas_df

In [5]:
# loading & preparing glmt data 
glmt_char_df = pd.read_csv(Path.joinpath(cset.path_MESMER_char, 'gmt_characteristics_thsld_150.csv'), index_col = 0)

# merge datasets into a single regression dataset
regression_df = gdp_df.loc[:, ['iso', 'scen_run', 'diff_SSP2']].merge(glmt_char_df, left_on='scen_run', right_index=True)
regression_df['GDP_baseline'] = gdp_baseline
regression_df = regression_df.merge(tas_df[['iso', 'scen_run', 'tas_soc']], on=['iso', 'scen_run'], how='left')
#   - add baseline temperature values
regression_df['tas_baseline'] = regression_df.groupby('iso')['tas_soc'].transform('mean')
regression_df['tas_soc']      -= regression_df['tas_baseline']
regression_df                 = regression_df.sort_values(['iso', 'scen_run'])
regression_df['gmt_os']       = regression_df['gmt_max']- regression_df['gmt_eoc']
regression_df['log_gmt_cum']  = np.log(regression_df['gmt_cum'])
regression_df['gmt_delta_eoc'] = 1.5+regression_df['gmt_exc']*2-regression_df['gmt_eoc']
regression_df['gmt_delta_max'] = regression_df['gmt_max']-(1.5+regression_df['gmt_exc']*2)
# clear storage
# del gdp_df, glmt_char_df, tas_df

In [6]:
regression_df['gmt_delta_eoc'] = regression_df['gmt_cum']-regression_df['gmt_eoc_exc']
regression_df['gmt_delta_max'] = regression_df['gmt_max_exc']-regression_df['gmt_cum']

In [23]:
np.corrcoef(regression_df['gmt_delta_eoc'], regression_df['gmt_delta_max'])

array([[1.        , 0.94026837],
       [0.94026837, 1.        ]])

In [8]:
# better target variable compared to regression_df['diff_SSP2']
y = np.log(regression_df['diff_SSP2'])

# Define predictor variables (excluding 'iso' and the target variable)
predictors = ['tas_soc', 'tas_baseline', 'gmt_cum', 'gmt_max', 'gmt_eoc', 'gmt_os', 'gmt_od', 'gmt_ud', 'gmt_exc', 'gmt_delta_eoc', 'gmt_delta_max', 'gmt_uxc', 'gmt_max_exc', 'gmt_eoc_exc']
# Generate second-order terms
poly       = PolynomialFeatures(2, interaction_only=False, include_bias=False)
X_poly     = poly.fit_transform(regression_df[predictors])
X_poly_std = StandardScaler().fit_transform(X_poly)
# Extract polynomial feature names
poly_feature_names  = poly.get_feature_names_out(predictors)
predictor_df        = pd.DataFrame(X_poly_std, columns = poly_feature_names)
regression_df.index = predictor_df.index
# interactions with tas_baseline 
interaction_terms   = [f for f in poly_feature_names if ('tas_baseline' in f) & (f != 'tas_baseline') & (f != 'tas_baseline^2')]
# Combine original predictors, interaction terms, and polynomial features 
all_predictors     = list(predictors) + list(interaction_terms) + list([f+'^2' for f in predictors])
# intercept matrix for country fixed effects  
n_countries = int(len(regression_df)/1000)
X_intercept = np.zeros((len(regression_df), n_countries))
for i in range(n_countries):
    X_intercept[i*1000:(i+1)*1000, i] = 1

### Step 2: Informed Model selection
- either: predictors are highly correlated --> Lasso Regression
- or: refine predictors to remove colinearity

In [9]:
predictor_sets = [
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_os', 'tas_baseline gmt_os'],
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_max', 'tas_baseline gmt_max', 'gmt_os', 'tas_baseline gmt_os'], 
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_od', 'tas_baseline gmt_od'], 
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_max', 'tas_baseline gmt_max', 'gmt_od', 'tas_baseline gmt_od'], 
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_cum'],
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_delta_eoc', 'tas_baseline gmt_delta_eoc'],
                    ['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_delta_eoc', 'tas_baseline gmt_delta_eoc', 'gmt_delta_max', 'tas_baseline gmt_delta_max'],
                    ]

In [11]:
sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_exc', 'tas_baseline gmt_exc']
country_data  = regression_df.copy()
country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
y_country = np.log(regression_df['diff_SSP2'].values)

X_country_raw = predictor_df.loc[:, sel_predictors].copy()
X_country     = X_country_raw.values.copy()
X_train       = X_country[country_data.run < 70,:].copy()
y_train       = y_country[country_data.run < 70].copy()
X_test        = X_country[country_data.run >= 70,:].copy()
y_test        = y_country[country_data.run >= 70].copy()

X_intercept_train = X_intercept[country_data.run < 70,:].copy()
X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

# model = sm.OLS(endog = y_train, 
#             exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
#                                                                         alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
#                                                                         L1_wt  = 0.5,
#                                                                         refit  = True)
            
# print(model.summary())
model = sm.OLS(endog = y_train, 
            exog = np.c_[X_train, X_intercept_train]).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.864
Model:                            OLS   Adj. R-squared (uncentered):              0.864
Method:                 Least Squares   F-statistic:                              4287.
Date:                Mon, 07 Oct 2024   Prob (F-statistic):                        0.00
Time:                        17:04:02   Log-Likelihood:                          3770.2
No. Observations:              194994   AIC:                                     -6964.
Df Residuals:                  194706   BIC:                                     -4032.
Df Model:                         288                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.4517      0.001   -391.065      0.000      -0.454      -0.449
x2             0.5738      0.001    495.874      0.000       0.572       0.576
x3             0.1584      0.002     70.253      0.000       0.154       0.163
x4            -0.3345      0.003   -125.848      0.000      -0.340      -0.329
x5             0.1908      0.003     68.722      0.000       0.185       0.196
x6            -0.3574      0.003   -131.444      0.000      -0.363      -0.352
x7            -0.3521      0.009    -38.677      0.000      -0.370      -0.334
x8            -0.3262      0.009    -36.019      0.000      -0.344      -0.308
x9            -0.3433      0.009    -38.680      0.000      -0.361      -0.326
x10           -0.3307      0.009    -35.791      0.000      -0.349      -0.313
x11           -0.3194      0.009    -35.918      0.000      -0.337      -0.302
x12           -0.3824      0.009    -41.237      0.000      -0.401      -0.364
x13           -0.4084      0.009    -45.479      0.000      -0.426      -0.391
x14           -0.2816      0.009    -30.464      0.000      -0.300      -0.263
x15           -0.2418      0.009    -26.713      0.000      -0.260      -0.224
x16           -0.2532      0.009    -27.494      0.000      -0.271      -0.235
x17           -0.2445      0.009    -26.771      0.000      -0.262      -0.227
x18           -0.2695      0.009    -30.383      0.000      -0.287      -0.252
x19           -0.2732      0.009    -29.359      0.000      -0.291      -0.255
x20           -0.2702      0.009    -29.939      0.000      -0.288      -0.252
x21           -0.2728      0.009    -29.059      0.000      -0.291      -0.254
x22           -0.2652      0.009    -29.449      0.000      -0.283      -0.248
x23           -0.2634      0.009    -28.494      0.000      -0.282      -0.245
x24           -0.2867      0.009    -31.654      0.000      -0.304      -0.269
x25           -0.3074      0.009    -33.528      0.000      -0.325      -0.289
x26           -0.2526      0.009    -27.864      0.000      -0.270      -0.235
x27           -0.2081      0.009    -23.239      0.000      -0.226      -0.191
x28           -0.2730      0.009    -29.202      0.000      -0.291      -0.255
x29           -0.3005      0.009    -33.426      0.000      -0.318      -0.283
x30           -0.2110      0.009    -22.636      0.000      -0.229      -0.193
x31           -0.2012      0.009    -22.544      0.000      -0.219      -0.184
x32           -0.2849      0.009    -30.686      0.000      -0.303      -0.267
x33           -0.2708      0.009    -29.752      0.000      -0.289      -0.253
x34           -0.2998      0.009    -32.905      0.000      -0.318      -0.282
x35           -0.2773      0.009    -30.370      0.000      -

In [10]:
sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_od', 'tas_baseline gmt_od', 'gmt_ud', 'tas_baseline gmt_ud']
country_data  = regression_df.copy()
country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
y_country = np.log(regression_df['diff_SSP2'].values)

X_country_raw = predictor_df.loc[:, sel_predictors].copy()
X_country     = X_country_raw.values.copy()
X_train       = X_country[country_data.run < 70,:].copy()
y_train       = y_country[country_data.run < 70].copy()
X_test        = X_country[country_data.run >= 70,:].copy()
y_test        = y_country[country_data.run >= 70].copy()

X_intercept_train = X_intercept[country_data.run < 70,:].copy()
X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

# model = sm.OLS(endog = y_train, 
#             exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
#                                                                         alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
#                                                                         L1_wt  = 0.5,
#                                                                         refit  = True)
            
# print(model.summary())
model = sm.OLS(endog = y_train, 
            exog = np.c_[X_train, X_intercept_train]).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.852
Model:                            OLS   Adj. R-squared (uncentered):              0.852
Method:                 Least Squares   F-statistic:                              3867.
Date:                Mon, 07 Oct 2024   Prob (F-statistic):                        0.00
Time:                        17:03:21   Log-Likelihood:                         -4286.5
No. Observations:              194994   AIC:                                      9153.
Df Residuals:                  194704   BIC:                                  1.211e+04
Df Model:                         290                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.4497      0.001   -373.568      0.000      -0.452      -0.447
x2             0.5732      0.001    475.196      0.000       0.571       0.576
x3             0.1481      0.002     59.268      0.000       0.143       0.153
x4            -0.6559      0.003   -216.144      0.000      -0.662      -0.650
x5            -0.0497      0.002    -28.313      0.000      -0.053      -0.046
x6             0.0249      0.002     13.496      0.000       0.021       0.029
x7             0.0800      0.003     23.417      0.000       0.073       0.087
x8             0.0301      0.003      8.897      0.000       0.023       0.037
x9            -0.4328      0.009    -45.646      0.000      -0.451      -0.414
x10           -0.3820      0.009    -40.382      0.000      -0.401      -0.363
x11           -0.2925      0.009    -31.598      0.000      -0.311      -0.274
x12           -0.3367      0.010    -34.943      0.000      -0.356      -0.318
x13           -0.3930      0.009    -42.374      0.000      -0.411      -0.375
x14           -0.3200      0.010    -33.135      0.000      -0.339      -0.301
x15           -0.2740      0.009    -29.216      0.000      -0.292      -0.256
x16           -0.3035      0.010    -31.495      0.000      -0.322      -0.285
x17           -0.2638      0.009    -27.952      0.000      -0.282      -0.245
x18           -0.3873      0.010    -40.418      0.000      -0.406      -0.369
x19           -0.3610      0.010    -37.807      0.000      -0.380      -0.342
x20           -0.3131      0.009    -33.845      0.000      -0.331      -0.295
x21           -0.3582      0.010    -36.961      0.000      -0.377      -0.339
x22           -0.4297      0.009    -45.572      0.000      -0.448      -0.411
x23           -0.4049      0.010    -41.327      0.000      -0.424      -0.386
x24           -0.4090      0.009    -43.471      0.000      -0.427      -0.391
x25           -0.2453      0.010    -25.452      0.000      -0.264      -0.226
x26           -0.2436      0.009    -25.790      0.000      -0.262      -0.225
x27           -0.4248      0.010    -44.508      0.000      -0.444      -0.406
x28           -0.3269      0.009    -34.419      0.000      -0.346      -0.308
x29           -0.0916      0.009     -9.797      0.000      -0.110      -0.073
x30           -0.1634      0.010    -16.767      0.000      -0.183      -0.144
x31           -0.1604      0.009    -17.090      0.000      -0.179      -0.142
x32           -0.1219      0.010    -12.531      0.000      -0.141      -0.103
x33           -0.1022      0.009    -10.981      0.000      -0.120      -0.084
x34           -0.3611      0.010    -37.354      0.000      -0.380      -0.342
x35           -0.3587      0.010    -37.706      0.000      -

In [33]:
regression_df['gmt_ud']/86

0         1.026963
1         0.744299
2         1.102907
3         0.744299
4         0.842162
            ...   
156995    0.117692
156996    0.092417
156997    0.092417
156998    0.318719
156999    0.142519
Name: gmt_ud, Length: 157000, dtype: float64

In [26]:
sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_exc', 'tas_baseline gmt_exc', 'gmt_uxc', 'tas_baseline gmt_uxc', 'gmt_os', 'tas_baseline gmt_os']
country_data  = regression_df.copy()
country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
y_country = np.log(regression_df['diff_SSP2'].values)

X_country_raw = predictor_df.loc[:, sel_predictors].copy()
X_country     = X_country_raw.values.copy()
X_train       = X_country[country_data.run < 70,:].copy()
y_train       = y_country[country_data.run < 70].copy()
X_test        = X_country[country_data.run >= 70,:].copy()
y_test        = y_country[country_data.run >= 70].copy()

X_intercept_train = X_intercept[country_data.run < 70,:].copy()
X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

# model = sm.OLS(endog = y_train, 
#             exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
#                                                                         alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
#                                                                         L1_wt  = 0.5,
#                                                                         refit  = True)
            
# print(model.summary())
model = sm.OLS(endog = y_train, 
            exog = np.c_[X_train, X_intercept_train]).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                 1.025e+04
Date:                Thu, 08 Aug 2024   Prob (F-statistic):               0.00
Time:                        17:52:09   Log-Likelihood:                 56346.
No. Observations:              108330   AIC:                        -1.124e+05
Df Residuals:                  108165   BIC:                        -1.108e+05
Df Model:                         164                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5425      0.001   -578.164      0.000      -0.544      -0.541
x2             0.6919      0.001    735.443      0.000       0.690       0.694
x3             0.2931      0.001    212.910      0.000       0.290       0.296
x4            -0.5255      0.001   -365.210      0.000      -0.528      -0.523
x5            -0.0810      0.001    -58.636      0.000      -0.084      -0.078
x6             0.1298      0.001     89.977      0.000       0.127       0.133
x7            -0.0144      0.001    -12.255      0.000      -0.017      -0.012
x8             0.0253      0.001     20.438      0.000       0.023       0.028
x9            -0.1710      0.006    -30.997      0.000      -0.182      -0.160
x10           -0.4541      0.005    -82.652      0.000      -0.465      -0.443
x11           -0.1818      0.006    -33.005      0.000      -0.193      -0.171
x12           -0.7270      0.006   -130.403      0.000      -0.738      -0.716
x13           -0.1853      0.005    -33.785      0.000      -0.196      -0.175
x14            0.0439      0.006      7.851      0.000       0.033       0.055
x15           -0.1841      0.005    -33.538      0.000      -0.195      -0.173
x16            0.0542      0.006      9.663      0.000       0.043       0.065
x17            0.0306      0.006      5.470      0.000       0.020       0.042
x18           -0.3766      0.005    -68.602      0.000      -0.387      -0.366
x19           -0.0492      0.006     -8.838      0.000      -0.060      -0.038
x20           -0.4588      0.006    -82.616      0.000      -0.470      -0.448
x21           -0.6150      0.006   -110.190      0.000      -0.626      -0.604
x22           -0.4020      0.006    -72.683      0.000      -0.413      -0.391
x23           -0.0810      0.006    -14.637      0.000      -0.092      -0.070
x24           -0.1103      0.006    -19.962      0.000      -0.121      -0.099
x25            0.1307      0.006     23.209      0.000       0.120       0.142
x26           -0.4221      0.006    -76.243      0.000      -0.433      -0.411
x27           -0.3967      0.005    -72.381      0.000      -0.407      -0.386
x28           -0.4248      0.006    -77.223      0.000      -0.436      -0.414
x29           -0.4058      0.006    -73.079      0.000      -0.417      -0.395
x30           -0.2651      0.005    -48.319      0.000      -0.276      -0.254
x31           -0.5174      0.005    -94.250      0.000      -0.528      -0.507
x32           -0.4551      0.006    -82.383      0.000      -0.466      -0.444
x33            0.2166      0.006     38.127      0.000       0.205       0.228
x34            0.0701      0.006     12.466      0.000       0.059       0.081
x35            0.0129      0.006      2.308      0.021       0.002       0.024
x36           -0.0943      0.006    -17.122      0.000      -0.105      -0.084
x3

In [16]:
sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_exc', 'tas_baseline gmt_exc', 'gmt_uxc', 'tas_baseline gmt_uxc', 'gmt_delta_eoc', 'tas_baseline gmt_delta_eoc', 'gmt_delta_max', 'tas_baseline gmt_delta_max']
country_data  = regression_df.copy()
country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
y_country = np.log(regression_df['diff_SSP2'].values)

X_country_raw = predictor_df.loc[:, sel_predictors].copy()
X_country     = X_country_raw.values.copy()
X_train       = X_country[country_data.run < 70,:].copy()
y_train       = y_country[country_data.run < 70].copy()
X_test        = X_country[country_data.run >= 70,:].copy()
y_test        = y_country[country_data.run >= 70].copy()

X_intercept_train = X_intercept[country_data.run < 70,:].copy()
X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

# model = sm.OLS(endog = y_train, 
#             exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
#                                                                         alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
#                                                                         L1_wt  = 0.5,
#                                                                         refit  = True)
            
# print(model.summary())
model = sm.OLS(endog = y_train, 
            exog = np.c_[X_train, X_intercept_train]).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                 1.025e+04
Date:                Thu, 08 Aug 2024   Prob (F-statistic):               0.00
Time:                        17:25:45   Log-Likelihood:                 56346.
No. Observations:              108330   AIC:                        -1.124e+05
Df Residuals:                  108165   BIC:                        -1.108e+05
Df Model:                         164                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5425      0.001   -578.164      0.000      -0.544      -0.541
x2             0.6919      0.001    735.443      0.000       0.690       0.694
x3             0.2931      0.001    212.910      0.000       0.290       0.296
x4            -0.5255      0.001   -365.210      0.000      -0.528      -0.523
x5            -0.0810      0.001    -58.636      0.000      -0.084      -0.078
x6             0.1298      0.001     89.977      0.000       0.127       0.133
x7            -0.0144      0.001    -12.255      0.000      -0.017      -0.012
x8             0.0253      0.001     20.438      0.000       0.023       0.028
x9            -0.1710      0.006    -30.997      0.000      -0.182      -0.160
x10           -0.4541      0.005    -82.652      0.000      -0.465      -0.443
x11           -0.1818      0.006    -33.005      0.000      -0.193      -0.171
x12           -0.7270      0.006   -130.403      0.000      -0.738      -0.716
x13           -0.1853      0.005    -33.785      0.000      -0.196      -0.175
x14            0.0439      0.006      7.851      0.000       0.033       0.055
x15           -0.1841      0.005    -33.538      0.000      -0.195      -0.173
x16            0.0542      0.006      9.663      0.000       0.043       0.065
x17            0.0306      0.006      5.470      0.000       0.020       0.042
x18           -0.3766      0.005    -68.602      0.000      -0.387      -0.366
x19           -0.0492      0.006     -8.838      0.000      -0.060      -0.038
x20           -0.4588      0.006    -82.616      0.000      -0.470      -0.448
x21           -0.6150      0.006   -110.190      0.000      -0.626      -0.604
x22           -0.4020      0.006    -72.683      0.000      -0.413      -0.391
x23           -0.0810      0.006    -14.637      0.000      -0.092      -0.070
x24           -0.1103      0.006    -19.962      0.000      -0.121      -0.099
x25            0.1307      0.006     23.209      0.000       0.120       0.142
x26           -0.4221      0.006    -76.243      0.000      -0.433      -0.411
x27           -0.3967      0.005    -72.381      0.000      -0.407      -0.386
x28           -0.4248      0.006    -77.223      0.000      -0.436      -0.414
x29           -0.4058      0.006    -73.079      0.000      -0.417      -0.395
x30           -0.2651      0.005    -48.319      0.000      -0.276      -0.254
x31           -0.5174      0.005    -94.250      0.000      -0.528      -0.507
x32           -0.4551      0.006    -82.383      0.000      -0.466      -0.444
x33            0.2166      0.006     38.127      0.000       0.205       0.228
x34            0.0701      0.006     12.466      0.000       0.059       0.081
x35            0.0129      0.006      2.308      0.021       0.002       0.024
x36           -0.0943      0.006    -17.122      0.000      -0.105      -0.084
x3

In [177]:
np.corrcoef(regression_df['gmt_max'], regression_df['gmt_exc'])

array([[1.        , 0.94139032],
       [0.94139032, 1.        ]])

In [163]:
regression_df['gmt_delta_eoc'].min()

-0.47103255813953515

In [164]:
regression_df['gmt_delta_eoc'].max()

1.3380767441860466

In [157]:
for sel_predictors in predictor_sets: 
    print(sel_predictors)
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    # model = sm.OLS(endog = y_train, 
    #             exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
    #                                                                         alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
    #                                                                         L1_wt  = 0.5,
    #                                                                         refit  = True)
                
    # print(model.summary())
    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit()
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('   - RMSE train:', rmse(y_train, model.fittedvalues), 'RMSE test:', rmse(y_test, y_test_pred))
    print('   - r2 train:', model.rsquared, 'r2 test:', r2_score(y_test, y_test_pred))

['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_os', 'tas_baseline gmt_os']
   - RMSE train: 0.15679351594371096 RMSE test: 0.16168212851703842
   - r2 train: 0.9281942657075883 r2 test: 0.9228905730008077
['tas_soc', 'tas_soc tas_baseline', 'gmt_max', 'tas_baseline gmt_max', 'gmt_os', 'tas_baseline gmt_os']
   - RMSE train: 0.15679351594371096 RMSE test: 0.1616821285170388
   - r2 train: 0.9281942657075883 r2 test: 0.9228905730008073
['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_od', 'tas_baseline gmt_od']
   - RMSE train: 0.16043534707538365 RMSE test: 0.16516250443447217
   - r2 train: 0.9248198745159371 r2 test: 0.919535121602071
['tas_soc', 'tas_soc tas_baseline', 'gmt_max', 'tas_baseline gmt_max', 'gmt_od', 'tas_baseline gmt_od']
   - RMSE train: 0.15718304994397705 RMSE test: 0.16194050508545293
   - r2 train: 0.9278370376716059 r2 test: 0.9226439262187451
['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_c

In [ ]:
# use ElasticNet Regression for overcoming issues /w multi-colinearity of driver variables
# cross-validation for finding the best regularization parameter 

In [71]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_max', 'tas_baseline gmt_max']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.5,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     5054.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:30:43   Log-Likelihood:                 20101.
No. Observations:              108330   AIC:                        -3.988e+04
Df Residuals:                  108170   BIC:                        -3.834e+04
Df Model:                         160                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5384      0.001   -410.909      0.0

In [80]:
sel_predictors = ['tas_soc', 'tas_soc tas_baseline',  'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_os', 'tas_baseline gmt_os']
country_data  = regression_df.copy()
country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
y_country = np.log(regression_df['diff_SSP2'].values)

X_country_raw = predictor_df.loc[:, sel_predictors].copy()
X_country     = X_country_raw.values.copy()
X_train       = X_country[country_data.run < 70,:].copy()
y_train       = y_country[country_data.run < 70].copy()
X_test        = X_country[country_data.run >= 70,:].copy()
y_test        = y_country[country_data.run >= 70].copy()

X_intercept_train = X_intercept[country_data.run < 70,:].copy()
X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

model = sm.OLS(endog = y_train, 
            exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                        alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                        L1_wt  = 0.5,
                                                                        refit  = True)
            
print(model.summary())

y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
print('RMSE test:', rmse(y_test, y_test_pred))
print('RMSE train:', rmse(y_train, model.fittedvalues))
print('r2 test:', r2_score(y_test, y_test_pred))
print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                 1.023e+04
Date:                Wed, 07 Aug 2024   Prob (F-statistic):               0.00
Time:                        09:32:16   Log-Likelihood:                 56220.
No. Observations:              108330   AIC:                        -1.121e+05
Df Residuals:                  108166   BIC:                        -1.105e+05
Df Model:                         164                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5433      0.001   -578.575      0.0

In [72]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_cum']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.5,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     5363.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:31:49   Log-Likelihood:                 22949.
No. Observations:              108330   AIC:                        -4.558e+04
Df Residuals:                  108170   BIC:                        -4.403e+04
Df Model:                         160                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5399      0.001   -423.014      0.0

In [70]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc', 'tas_baseline gmt_eoc']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.5,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     6997.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:29:59   Log-Likelihood:                 36229.
No. Observations:              108330   AIC:                        -7.213e+04
Df Residuals:                  108169   BIC:                        -7.058e+04
Df Model:                         161                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5383      0.001   -476.750      0.0

In [140]:
regression_df['gmt_delta_eoc'].max()

0.2574534883720927

In [139]:
regression_df['gmt_delta_eoc'].min()

0.0

In [138]:
regression_df['gmt_delta_max'].max()


0.14274302325581356

In [137]:
regression_df['gmt_delta_max'].min()

0.0

In [148]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_eoc_exc', 'tas_baseline gmt_eoc_exc', 'gmt_delta_eoc', 'tas_baseline gmt_delta_eoc', 'gmt_delta_max', 'tas_baseline gmt_delta_max']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.8,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     9818.
Date:                Wed, 07 Aug 2024   Prob (F-statistic):               0.00
Time:                        11:34:19   Log-Likelihood:                 54442.
No. Observations:              108330   AIC:                        -1.086e+05
Df Residuals:                  108165   BIC:                        -1.070e+05
Df Model:                         165                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5442      0.001   -569.943      0.0

In [114]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_cum_eoc_diff', 'tas_baseline gmt_cum_eoc_diff']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.5,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     5735.
Date:                Wed, 07 Aug 2024   Prob (F-statistic):               0.00
Time:                        10:02:35   Log-Likelihood:                 26491.
No. Observations:              108330   AIC:                        -5.266e+04
Df Residuals:                  108169   BIC:                        -5.110e+04
Df Model:                         161                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5416      0.001   -438.354      0.0

In [ ]:
    sel_predictors = predictors_sets[5]
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.8,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

In [73]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline',  'gmt_eoc', 'tas_baseline gmt_eoc', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_cum gmt_eoc']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.8,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     8350.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:50:00   Log-Likelihood:                 45340.
No. Observations:              108330   AIC:                        -9.035e+04
Df Residuals:                  108168   BIC:                        -8.879e+04
Df Model:                         162                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5414      0.001   -521.579      0.0

In [76]:
    sel_predictors = ['tas_soc', 'tas_soc tas_baseline',  'gmt_max', 'tas_baseline gmt_max', 'gmt_cum', 'tas_baseline gmt_cum', 'gmt_cum gmt_max']
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.02]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.5,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     9927.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:53:20   Log-Likelihood:                 54072.
No. Observations:              108330   AIC:                        -1.078e+05
Df Residuals:                  108168   BIC:                        -1.063e+05
Df Model:                         162                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5394      0.001   -563.272      0.0

In [68]:
    sel_predictors = predictors_sets[6]
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.8,
                                                                            refit  = True)
                
    print(model.summary())
    
    y_test_pred = model.predict(np.c_[X_test, X_intercept_test])
    print('RMSE test:', rmse(y_test, y_test_pred))
    print('RMSE train:', rmse(y_train, model.fittedvalues))
    print('r2 test:', r2_score(y_test, y_test_pred))
    print('r2 train:', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     9927.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:27:01   Log-Likelihood:                 54072.
No. Observations:              108330   AIC:                        -1.078e+05
Df Residuals:                  108168   BIC:                        -1.063e+05
Df Model:                         162                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5394      0.001   -563.272      0.0

In [48]:
    sel_predictors = predictors_sets[8]
    country_data  = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y_country = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

    model = sm.OLS(endog = y_train, 
                exog = np.c_[X_train, X_intercept_train]).fit_regularized(method = 'elastic_net', 
                                                                            alpha  = [0.01]*len(sel_predictors)+[0]*n_countries,
                                                                            L1_wt  = 0.8,
                                                                            refit  = True)
                
    print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     9927.
Date:                Tue, 06 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:10:53   Log-Likelihood:                 54072.
No. Observations:              108330   AIC:                        -1.078e+05
Df Residuals:                  108168   BIC:                        -1.063e+05
Df Model:                         162                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5394      0.001   -563.272      0.0

In [ ]:
for sel_predictors in predictors_sets:
    country_data = regression_df.copy()
    country_data.loc[:, 'run'] = [int(f.split('_')[1]) for f in country_data['scen_run'].values]
    y = np.log(regression_df['diff_SSP2'].values)

    X_country_raw = predictor_df.loc[:, sel_predictors].copy()
    X_country     = X_country_raw.values.copy()
    X_train       = X_country[country_data.run < 70,:].copy()
    y_train       = y_country[country_data.run < 70].copy()
    X_test        = X_country[country_data.run >= 70,:].copy()
    y_test        = y_country[country_data.run >= 70].copy()

    X_intercept_train = X_intercept[country_data.run < 70,:].copy()
    X_intercept_test  = X_intercept[country_data.run >= 70,:].copy()

In [82]:
np.corrcoef(regression_df['gmt_od'], regression_df['gmt_os'])

array([[1.        , 0.98307142],
       [0.98307142, 1.        ]])

In [98]:
np.corrcoef(regression_df['gmt_cum_eoc_diff'], regression_df['gmt_cum'])

array([[1.        , 0.99996138],
       [0.99996138, 1.        ]])

In [99]:
regression_df['gmt_cum_eoc_diff']

0         183.0947
1         140.9423
2         186.1600
3         140.9423
4         149.6451
            ...   
156995    158.2183
156996    155.3633
156997    155.3633
156998    196.9180
156999    180.8987
Name: gmt_cum_eoc_diff, Length: 157000, dtype: float64

In [92]:
np.corrcoef(regression_df['gmt_od'][200:300], regression_df['gmt_eoc'][200:300])

array([[ 1.        , -0.31443778],
       [-0.31443778,  1.        ]])

In [85]:
regression_df

,iso,scen_run,diff_SSP2,gmt_ntwr,gmt_mtwr,gmt_ltwr,gmt_max,gmt_eoc,gmt_soc,gmt_cum,...,gmt_exc,gmt_uxc,gmt_frac_os,gmt_cum_exc,gmt_cum_neg,model_id,GDP_baseline,tas_soc,tas_baseline,gmt_os
0,AFG,CurPol_1,0.881178,0.238996,0.725960,1.463975,3.1931,3.1931,1.2728,2.166137,...,0.681501,0.015364,0.860465,2.166137,0.000000,CNRM-CM6-1-HR,1347.681378,0.655253,13.633718,0.0000
1,AFG,CurPol_10,0.985876,0.165944,0.541180,1.082506,2.4112,2.4112,1.0082,1.666901,...,0.269847,0.102945,0.593023,1.666901,0.000000,NESM3,5845.591242,1.318173,13.633718,0.0000
2,AFG,CurPol_100,0.898922,0.225488,0.723412,1.517781,3.3060,3.3060,1.2919,2.203093,...,0.717580,0.014487,0.860465,2.203093,0.000000,MCM-UA-1-0,8231.241656,0.812160,13.633718,0.0000
3,AFG,CurPol_11,0.884318,0.165944,0.541180,1.082506,2.4112,2.4112,1.0082,1.666901,...,0.269847,0.102945,0.593023,1.666901,0.000000,NorESM2-LM,5845.591242,-0.064920,13.633718,0.0000
4,AFG,CurPol_12,0.896899,0.198676,0.608404,1.217431,2.6126,2.6126,1.0262,1.770438,...,0.352329,0.081891,0.651163,1.770438,0.000000,FGOALS-g3,5845.591242,0.482229,13.633718,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156995,ZWE,ssp534-over_95,0.333189,0.325440,1.037868,0.926525,2.2254,1.8069,1.0766,1.860758,...,0.403819,0.043060,0.813953,1.954394,0.093636,CESM2,498.120595,-0.207576,21.798667,0.4185
156996,ZWE,ssp534-over_96,0.396659,0.278296,0.915208,0.774850,2.1576,1.7505,1.1556,1.826905,...,0.361413,0.034508,0.813953,1.922951,0.096047,CESM2,498.120595,-0.252657,21.798667,0.4071
156997,ZWE,ssp534-over_97,1.002097,0.278296,0.915208,0.774850,2.1576,1.7505,1.1556,1.826905,...,0.361413,0.034508,0.813953,1.922951,0.096047,NESM3,498.120595,0.814703,21.798667,0.4071
156998,ZWE,ssp534-over_98,0.431720,0.410116,1.363168,1.507878,2.7541,2.5757,1.1730,2.319694,...,0.841434,0.021740,0.872093,2.350341,0.030647,AWI-CM-1-1-MR,498.120595,0.189254,21.798667,0.1784
